# EIANN Tutorial 1:  
# Feedforward ANN trained with backprop

In [2]:
import EIANN.EIANN as eiann

/Users/ag1880/github-repos/Milstein-Lab/EIANN/EIANN/utils/data_utils.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Build and train a simple feedforward neural network with EIANN

To build a neural network using the EIANN library, you need to specify 3 things:
1. The number of neurons in each layer and sub-population
2. The projections (how the populations connect to each other)
3. Global parameters that we want to apply to the whole network (e.g. learning rate)

The EIANN interface is designed to have all of these things specified through python dicts. A network with the specified architecture will then be built using pytorch layers as the backend building blocks. 

There are two easy ways to create the python dicts that will specify the network architecture:

1. Programmatically using the NetworkBuilder class
2. Using a .yaml configuration file (you can find examples of these in the network_config directory)

The first method is convenient for quickly prototyping and testing different network architectures.  The second method is particularly useful for hyperparameter optimization and experiment reproducibility.

Here we will show you a simple example with both methods.

### Programming interface

In EIANN, network architecture is defined by: 1. creating populations of neurons, and 2. specifying how they are connected. This approach is flexible and allows you to recurrently connect any two populations, regardless of layer hierarchy.

In [3]:
network_config = eiann.NetworkBuilder()

# Define layers and populations
network_config.layer('Input').population('E', size=784)
network_config.layer('H1').population('E', size=500, activation='relu')
network_config.layer('Output').population('E', size=10, activation='softmax')

# Create connections between populations
network_config.connect(source='Input.E', target='H1.E')
network_config.connect(source='H1.E', target='Output.E')

# Build the network
network_config.print_architecture()

Network Architecture:
Input.E (784) -> H1.E (500): No learning rule
H1.E (500) -> Output.E (10): No learning rule



Rather than defining only the weight matrix dimensions, this population-based syntax allows you to also specify a number of attributes for each population. For example, if you want the network to respect Dale's Law, you can specify whether each population is excitatory or inhibitory. Doing this will automatically bound the sign of outgoing weights, both at initialization and during training. 

In [7]:
network_config.layer('H1').population('E', 500, 'relu').type('Exc') 
network_config.layer('H1').population('SomaI', 50, 'relu').type('Inh') 

AttributeError: 'LayerBuilder' object has no attribute 'type'

Should you prefer, these weigtht bounds can also be set for each individual projection (weight matrix)

In [6]:
network_config.connect(source='H1.E', target='H1.SomaI').type('Exc') 
network_config.connect(source='H1.SomaI', target='H1.E').type('Inh') 

Once we have specified all the populations and projections that define the network architecture, we can use the `build_network` function to create the neural network (pytorch) object.